In [ ]:
%pip install scipy --upgrade
import math
import matplotlib.pyplot as plt
import scipy.stats as stats
import scipy.optimize as opt
import numpy as np
import random

%pip install gurobipy
import gurobipy as grb

     |████████████████████████████████| 38.1 MB 2.0 MB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.
     |████████████████████████████████| 11.5 MB 7.7 MB/s 


### Q1-1

In [ ]:
#inputs
cost=np.array([250,650,250,500,700,30,350,70])
p=np.array([0.9,0.7,0.6,0.4,0.8,0.6,0.7,0.9])
revpar=np.array([[600,750,900],[1250,1500,1600],[500,600,750],[1600,1800,1900],
                 [1150,1200,1400],[150,180,250],[750,900,1000],[220,250,320]])
print(revpar.shape)
print(revpar[1,:])

(8, 3)
[1250 1500 1600]


In [ ]:
np.random.seed(5566)
#I: The number of projects
I=8
#S: The number of simulation runs
S=10000
simsuccess=np.zeros((S,I))
simrev=np.zeros((S,I))
avgrev=np.zeros(I)
for i in range(I):
  simsuccess[0:S,i]=np.random.binomial(1,p[i],S)
  simrev[:,i]=np.random.triangular(revpar[i,0],revpar[i,1],revpar[i,2],S)
  simrev=np.multiply(simrev,simsuccess)

# print(simsuccess)
# print(simrev)
# print(simrev.shape)

In [ ]:
# profit of all projects are selected
x_vars = [1, 1, 1, 1, 1, 1, 1, 1]

sum = 0
for i in range(I):
  for s in range(S):
    sum += x_vars[i]*(simrev[s,i]-cost[i])

sum / S

1929.5946688591935

### Q1-2

In [ ]:
# success rate of (1,2,5,6,7)project are selected
x_vars = np.array([1, 1, 0, 0, 1, 1, 1, 0])

print(round(np.sum(x_vars*p), 1))

3.7


In [ ]:
# success rate of (1,2,4,5,6,7)project are selected
x_vars = [1, 1, 0, 1, 1, 1, 1, 0]

print(round(np.sum(x_vars*p), 1))

4.1


In [ ]:
# profit of (1,2,5,6,7)project are selected
x_vars = [1, 1, 0, 0, 1, 1, 1, 0]

sum = 0
for i in range(I):
  for s in range(S):
    sum += x_vars[i]*(simrev[s,i]-cost[i])

sum / S

1455.7248956016815

In [ ]:
# profit of (1,2,4,5,6,7)project are selected
x_vars = [1, 1, 0, 1, 1, 1, 1, 0]

sum = 0
for i in range(I):
  for s in range(S):
    sum += x_vars[i]*(simrev[s,i]-cost[i])

sum / S

#project(1,2,4,5,6,7)的利益大於project(1,2,5,6,7)。此外project(1,2,4,5,6,7)的成功率也大於project(1,2,5,6,7)。
#我會選擇project(1,2,5,6,7)因為有較高的獲利能力

1649.8025501942568

### Q2-1

In [ ]:
p = [0.03, 0.05, 0.07, 0.09, 0.11, 0.15, 0.18, 0.14, 0.08, 0.05, 0.03, 0.02]

In [ ]:
np.random.seed(7788)
S = 5000

distribution_of_seat_num = np.array(range(14,26,1))
sim_demand = np.random.choice(distribution_of_seat_num, size=S, p=p)

In [ ]:
# try to maximize profit
def revenue(n):
  profits = np.zeros(S)
  Eprofit = 0
  for s in range(S):
      d = sim_demand[s]
      reserved = min(d, n)
      show = np.random.binomial(reserved, 0.9, 1)
      profits[s] = 150*reserved - 325*max(0, show - 19)

  #print(profits)
  Eprofit=round(np.mean(profits))

  return -Eprofit

res = opt.minimize(revenue, 14, bounds=[(14, 26)])
print("seats:", res.x[0])
print("profit: ", -res.fun)

seats: 21.0
profit:  2793.0


### Q2-2

In [ ]:
# try to minimize cost of over-sell
def loss(n):
  profits_loss = np.zeros(S)
  Eprofit = 0
  for s in range(S):
      d = sim_demand[s]
      reserved = min(d, n)
      show = np.random.binomial(reserved, 0.9, 1)
      profits_loss[s] = - 325*max(0, show - 19)

  #print(profits_loss)
  Eprofit=round(np.mean(profits_loss))

  return -Eprofit

res = opt.minimize(loss, 14, bounds=[(14, 26)])
print("seats:", res.x[0])

seats: 19.0


### Q2-3
Q2-1 的目標函數是最大化期望值，因此有可能獲得更大的利潤。 但同時風險大，賠錢的概率和損失也較大。<br>

Q2-2的目標函數是最大化成功率，也就是最小化風險，所以賠錢的概率小，損失也少，但是期望值會比較低，可能的最大利潤不是 高的。<br>

###Q3-1

In [ ]:
consultantY=80000
threshold=5000
fixedC=3995
variable_cost_meal=11
S=1000
mealSold_mean=3000
mealSold_std=1000
mealSold=np.random.normal(mealSold_mean,mealSold_std,S)
labor_low=5040
labor_upper=6860
laborCost=np.random.uniform(labor_low,labor_upper,S)
mealPrice=np.array([20,18.5,16.5,15])
mealPriceP=np.array([0.25,0.35,0.3,0.1])
simMealPrice=np.random.choice(mealPrice,S,p=mealPriceP)



part_minimum=3500
part_thres=9000
profits_nopat=np.empty(S)
profits_pat=np.empty(S)
for s in range(S):
  profits_nopat[s]=mealSold[s]*(simMealPrice[s]-variable_cost_meal)-laborCost[s]-fixedC
  if(profits_nopat[s]<part_minimum):
    profits_pat[s]=part_minimum
  elif(profits_nopat[s]>part_thres):
    profits_pat[s]=(profits_pat[s]-part_thres)*0.1+part_thres
  else:
    profits_pat[s]=profits_nopat[s]

avgprofit_nopat=np.mean(profits_nopat)
print("Without PartnerShip")
print("Average: ")
print(avgprofit_nopat)
print("Max: ")
print(max(profits_nopat))
print("Min: ")
print(min(profits_nopat))

# 無法保證獲利，會有虧損的情況產生
#不繼續經營產生的虧損甚至可能比較小

Without PartnerShip
Average: 
10974.842657083966
Max: 
44353.42232937872
Min: 
-9029.011471258182


###Q3-2

In [ ]:
avgprofit_pat=np.mean(profits_pat)
print("With PartnerShip")
print("Average: ")
print(avgprofit_pat)
print("Max: ")
print(max(profits_pat))
print("Min: ")
print(min(profits_pat))

# 如果加入合夥人的話，你可能的最大利潤會降低，但是可以確保最低利潤為3500。
# 因此，若要追求高利潤，不適合找合夥人；但若追求穩定收入，就適合找合夥人



With PartnerShip
Average: 
6676.972955704596
Max: 
8977.002195329143
Min: 
3500.0


###Q3-3

In [ ]:
#estimate P(profits of no partnership > profits of partnership)
NoPatMore=np.empty (S)

mealProfit=np.array([9,7.5,5.5,4])
mealProfitP=np.array([0.25,0.35,0.3,0.1])
P = 0
for i in range(4):
  for s in range(S):
    if(mealSold[s]*mealProfit[i]>12995 + profits_pat[s]):
      NoPatMore[s]=1
    else:
      NoPatMore[s]=0
    p=np.count_nonzero(NoPatMore)/S
  P = P + p*mealProfitP[i]
print(P)


###Q3-4

In [ ]:
Year_consultantY=80000
Year_threshold=5000*12
Year_fixedC=3995*12
Year_variable_cost_meal=11
S=1000
Year_mealSold_mean=3000*12
Year_mealSold_std=1000
Year_mealSold=np.random.normal(Year_mealSold_mean,Year_mealSold_std,S)
Year_labor_low=5040*12
Year_labor_upper=6860*12
Year_laborCost=np.random.uniform(Year_labor_low,Year_labor_upper,S)
Year_mealPrice=np.array([20,18.5,16.5,15])
Year_mealPriceP=np.array([0.25,0.35,0.3,0.1])
Year_simMealPrice=np.random.choice(Year_mealPrice,S,p=Year_mealPriceP)



Year_part_minimum=3500*12
Year_part_thres=9000*12
Year_profits_nopat=np.empty(S)
Year_profits_pat=np.empty(S)
for s in range(S):
  Year_profits_nopat[s]=Year_mealSold[s]*(Year_simMealPrice[s]-Year_variable_cost_meal)-Year_laborCost[s]-Year_fixedC
  if(Year_profits_nopat[s]<Year_part_minimum):
    Year_profits_pat[s]=Year_part_minimum
  elif(Year_profits_nopat[s]>Year_part_thres):
    Year_profits_pat[s]=(Year_profits_pat[s]-Year_part_thres)*0.1+Year_part_thres
  else:
    Year_profits_pat[s]=Year_profits_nopat[s]

Year_avgprofit_nopat=np.mean(Year_profits_nopat)
print("Without PartnerShip")
print("Average: ")
print(Year_avgprofit_nopat)
print("Max: ")
print(max(Year_profits_nopat))
print("Min: ")
print(min(Year_profits_nopat))

Without PartnerShip
Average: 
130295.96115709389
Max: 
244639.3535397034
Min: 
9179.202343216486


In [ ]:
Year_avgprofit_pat=np.mean(Year_profits_pat)
print("With PartnerShip")
print("Average: ")
print(Year_avgprofit_pat)
print("Max: ")
print(max(Year_profits_pat))
print("Min: ")
print(min(Year_profits_pat))

# 從穩定性（平均值）來看選擇不合作皆較有利，故以年來看會選擇不合作
#從最高獲利能力（最大值）來看選擇不合作皆較有利，故以年來看會選擇不合作


With PartnerShip
Average: 
86011.04045903304
Max: 
100369.31729486794
Min: 
42000.0
